In [ ]:
!pip install scikeras
!pip install flask-ngrok

In [ ]:
!pip install pyngrok
!ngrok authtoken "2VgRIx5A9ZHEX0wrF2ze7kTDlvy_5PizugQJbrkVEckpvmzcB"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=2bb12e1f3f918862ed5c42f29c9bf7a60bfa7f32e32334c2c71a6678910b5ab1
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# from google.colab import drive
#drive.mount('/drive')

In [ ]:
cd /content/drive/MyDrive/Colab/Modal

/content/drive/MyDrive/Colab/Modal


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import pickle
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template

In [ ]:
def twoLayerFeedForward():
    clf = Sequential()
    clf.add(Dense(64, activation='relu', input_dim=10))
    clf.add(Dropout(rate=0.2))
    clf.add(Dense(32, activation='relu'))
    clf.add(Dense(1, activation='sigmoid'))
    clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    return clf

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
  return render_template('index.html')

@app.route('/getprediction', methods=['POST'])
def getprediction():
  input = [float(x) if x.isnumeric() else x for x in request.form.values()]

  preprocessors = pickle.load(open('updated-preprocessors.pkl', 'rb'))
  model = pickle.load(open('upd_trained_ensemble.pkl', 'rb'))
  le = preprocessors['le']
  sc = preprocessors['sc']

  column = ['gender','ever_married','work_type','Residence_type','smoking_status']
  feature_names = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status']

  sample_data = np.array([input])
  sample_data_with_names = pd.DataFrame(sample_data, columns=feature_names)

  t = le.transform(sample_data_with_names[column].values.ravel())

  sample_data_with_names.iloc[0]['gender'] = t[0]
  sample_data_with_names.iloc[0]['ever_married'] = t[1]
  sample_data_with_names.iloc[0]['work_type'] = t[2]
  sample_data_with_names.iloc[0]['Residence_type'] = t[3]
  sample_data_with_names.iloc[0]['smoking_status'] = t[4]

  sample_data_with_names = sc.transform(sample_data_with_names)
  sample_data_with_names = pd.DataFrame(sample_data_with_names, columns=feature_names)

  print(sample_data_with_names)

  result = 'Yes' if model.predict(sample_data_with_names)[0] == 1 else 'No'

  return render_template('index.html', output='Prediction result from Model: {}'.format(result))

In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://565c-35-199-169-53.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Sep/2023 16:30:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Sep/2023 16:30:56] "GET /favicon.ico HTTP/1.1" 404 -


     gender      age  hypertension  heart_disease  ever_married  work_type  \
0  1.186997  0.96298      3.043196      -0.238947      0.723884  -0.161586   

   Residence_type  avg_glucose_level       bmi  smoking_status  
0         0.98408           0.747636  3.996723         1.07452  


INFO:werkzeug:127.0.0.1 - - [23/Sep/2023 16:31:31] "POST /getprediction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Sep/2023 16:36:02] "POST /getprediction HTTP/1.1" 200 -


     gender       age  hypertension  heart_disease  ever_married  work_type  \
0  1.186997  1.537935     -0.328602       4.185032      0.723884  -0.161586   

   Residence_type  avg_glucose_level       bmi  smoking_status  
0         0.98408           2.510921  3.996723         1.07452  
